<a href="https://colab.research.google.com/github/wjdolan/DS_Portfolio/blob/main/PetroleumDCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Petroleum Well DCA

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime

In [ ]:
# load data 

filepath = 'content/production_data'


df = pd.read_csv(filepath, parse_dates='')


t = df['Date']
q = df['Rate (SCF/d)']

df.head()

In [ ]:
# normalize the datetime data

timedelta = [j-i for i, j in zip(t[:-1], t[1:])]
timedelta = np.array(timedelta)
timedelta = timedelta / datetime.timedelta(days=1)


# take cumulative sum over timedeltas
t = np.cumsum(timedelta)
t = np.append(0, t)
t = t.astype(float)

In [ ]:
# plot the data

plt.plot(t, q, '.', color='red')
plt.title('Production Rate from 01/12/1990 to 01/11/1992', size=13, pad=15)
plt.xlabel('Days')
plt.ylabel('Rate (SCF/d)')
plt.xlim(xmin=0); plt.ylim(ymin=0)


plt.show()

In [ ]:
# normalize the time and rate data

t_normalized = t / max(t)
q_normalized = q / max(q)

In [ ]:
# function for hyperbolic decline

def hyperbolic(t, qi, di, b):
  return qi / (np.abs((1 + b * di * t))**(1/b))


# fitting the data with the hyperbolic function

from scipy.optimize import curve_fit


popt, pcov = curve_fit(hyperbolic, t_normalized, q_normalized)

In [ ]:
qi, di, b = popt


# de-normalize qi and di
qi = qi * max(q)
di = di / max(t)


print('Initial production rate:', np.round(qi, 3), 'MMSCF')
print('Initial decline rate:', np.round(di, 3), 'SCF/D')
print('Decline coefficient:', np.round(b, 3))

In [ ]:
# function for hyperbolic cumulative production

def cumpro(q_forecast, qi, di, b):
  return (((qi**b) / ((1 - b) * di)) * ((qi ** (1 - b)) - (q_forecast ** (1 - b)))) 
  

In [ ]:
# forecast gas rate until 1,500 days
t_forecast = np.arange(1501)
q_forecast = hyperbolic(t_forecast, qi, di, b)


# forecast cumulative production until 1,500 days
Qp_forecast = cumpro(q_forecast, qi, di, b)

In [ ]:
# plot the production data with the forecasts (rate and cum. production)
plt.figure(figsize=(15,5))


plt.subplot(1,2,1)
plt.plot(t, q, '.', color='red', label='Production Data')
plt.plot(t_forecast, q_forecast, label='Forecast')
plt.title('Gas Production Rate Result of DCA', size=13, pad=15)
plt.xlabel('Days')
plt.ylabel('Rate (SCF/d)')
plt.xlim(xmin=0); plt.ylim(ymin=0)
plt.legend()


plt.subplot(1,2,2)
plt.plot(t_forecast, Qp_forecast)
plt.title('Gas Cumulative Production Result of DCA', size=13, pad=15)
plt.xlabel('Days')
plt.ylabel('Production (SCF)')
plt.xlim(xmin=0); plt.ylim(ymin=0)


plt.show()